# YAMNet
+ tensorflow 공식 메뉴얼
+ https://www.tensorflow.org/tutorials/audio/transfer_learning_audio

## [선행 과정]
+ mp3 -> wav 파일 변환
+ csv 파일 통합 파일

In [47]:
!pip install "tensorflow==2.10.*"
# tensorflow_io 0.27 is compatible with TensorFlow 2.10
!pip install "tensorflow_io==0.27.*"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio

In [49]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [50]:
%cd '/content/gdrive/MyDrive/기업연계프로젝트/Music(Confirmed)/YAMNet'

/content/gdrive/MyDrive/기업연계프로젝트/Music(Confirmed)/YAMNet


In [51]:
# Utility functions for loading audio files and making sure the sample rate is correct.
# wav 파일 불러오기
@tf.function
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    
    return wav

In [52]:
# 데이터 통합
# import pandas as pd
# song = pd.read_csv('C:/music_data/Song.csv')
# station_song = pd.read_csv('C:/music_data/Station_Song.csv')
# station_tag = pd.read_csv('C:/music_data/Station_Tag.csv')
# station_tag_info = pd.read_csv('C:/music_data/Station_Tag_Info.csv')
# song.drop(columns = ['IDX'], inplace = True)
# station_song.drop(columns = ['IDX'], inplace = True)
# station_tag.drop(columns = ['IDX'], inplace = True)
# station_tag_info.columns = ['ST_TAG_ID','TYPE_ID','TAG_NAME','CDATE']
# print(song.columns, station_song.columns ,station_tag.columns, station_tag_info.columns, sep = '\n')
# part1 = pd.merge(song,station_song, on = 'SONG_ID', how = 'inner')
# part2 = pd.merge(part1,station_tag, on = 'ST_ID', how = 'inner')
# final = pd.merge(part2,station_tag_info, on = 'ST_TAG_ID', how = 'inner')
# final.sort_values(by = ['SONG_ID'], inplace = True)
# final_data = final.drop_duplicates(['SONG_ID']).reset_index(drop = True)
# final_data.head()
# final_data.to_csv('total_data.csv', index = False, encoding = 'utf-8')

In [53]:
final_data = pd.read_csv('./total_data.csv', encoding = 'utf-8')
final_data

,SONG_ID,SONG_TITLE,ST_ID,PRIORITY,ST_TAG_ID,TYPE_ID,TAG_NAME,CDATE
0,2466,오늘도 난,467710,2,5711,2,편안해요,2022-03-17 14:48:33
1,5112,끝 (End),456206,17,5766,12,신남,2022-03-31 14:31:33
2,7712,당신은 모르실거야 (CF - LIG 핑클편),488851,11,5322,2,화나요,2021-06-08 12:07:54
3,9270,매직 카펫 라이드,363115,13,5330,2,행복해요,2021-06-08 12:07:54
4,9302,Deep In The Night,489238,9,5322,2,화나요,2021-06-08 12:07:54
...,...,...,...,...,...,...,...,...
1233,34850634,사랑인가 봐,466008,1,5711,2,편안해요,2022-03-17 14:48:33
1234,34890162,다시 만날까 봐,466884,20,5711,2,편안해요,2022-03-17 14:48:33
1235,34927767,정이라고 하자 (Feat. 10CM),493585,12,5322,2,화나요,2021-06-08 12:07:54
1236,34943312,나의 X에게,493585,19,5322,2,화나요,2021-06-08 12:07:54


In [54]:
# pretrained yamnet load
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

In [55]:
# 감정 라벨링
my_classes = ['화나요','행복해요','편안해요','슬픔','불안','신남']
map_class_to_id = {'화나요':0,'행복해요':1,'편안해요':2,'슬픔':3,'불안':4,'신남':5}

class_id = final_data['TAG_NAME'].apply(lambda name : map_class_to_id[name])
filtered_pd = final_data.assign(TAG_NAME = class_id)

In [56]:
# wav 파일 주소 삽입
full_path = filtered_pd['SONG_ID'].apply(lambda row: '/content/gdrive/MyDrive/기업연계프로젝트/Music(Confirmed)/YAMNet/data_wav/'+ str(row) + '.wav')
filtered_pd = filtered_pd.assign(SONG_ID = full_path)
filtered_pd.head()

,SONG_ID,SONG_TITLE,ST_ID,PRIORITY,ST_TAG_ID,TYPE_ID,TAG_NAME,CDATE
0,/content/gdrive/MyDrive/기업연계프로젝트/Music(Confirm...,오늘도 난,467710,2,5711,2,2,2022-03-17 14:48:33
1,/content/gdrive/MyDrive/기업연계프로젝트/Music(Confirm...,끝 (End),456206,17,5766,12,5,2022-03-31 14:31:33
2,/content/gdrive/MyDrive/기업연계프로젝트/Music(Confirm...,당신은 모르실거야 (CF - LIG 핑클편),488851,11,5322,2,0,2021-06-08 12:07:54
3,/content/gdrive/MyDrive/기업연계프로젝트/Music(Confirm...,매직 카펫 라이드,363115,13,5330,2,1,2021-06-08 12:07:54
4,/content/gdrive/MyDrive/기업연계프로젝트/Music(Confirm...,Deep In The Night,489238,9,5322,2,0,2021-06-08 12:07:54


In [57]:
# 독립 변수와 종속 변수 설정
filenames = filtered_pd['SONG_ID']
targets = filtered_pd['TAG_NAME']

In [58]:
# from tensorflow.keras.utils import to_categorical
# targets_1hot = to_categorical(targets, num_classes = len(my_classes))

In [59]:
# train, valid, test 분할 (75 : 15 : 15)
from sklearn.model_selection import train_test_split
train_X, test_X , train_y, test_y = train_test_split(filenames, targets, test_size = 0.15)
train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size = 0.176)

In [60]:
# 데이터셋(tf.data.Dataset) 객체
# train
# trainX_ds = tf.data.Dataset.from_tensor_slices(train_X)
# trainy_ds = tf.data.Dataset.from_tensor_slices(train_y)
# train_ds = tf.data.Dataset.zip((trainX_ds, trainy_ds))
# valid
# validX_ds = tf.data.Dataset.from_tensor_slices(valid_X)
# validy_ds = tf.data.Dataset.from_tensor_slices(valid_y)
# valid_ds = tf.data.Dataset.zip((validX_ds, validy_ds))
# test
# testX_ds = tf.data.Dataset.from_tensor_slices(test_X)
# testy_ds = tf.data.Dataset.from_tensor_slices(test_y)
# test_ds = tf.data.Dataset.zip((testX_ds, testy_ds))
train_ds = tf.data.Dataset.from_tensor_slices((train_X, train_y))
valid_ds = tf.data.Dataset.from_tensor_slices((valid_X, valid_y))
test_ds = tf.data.Dataset.from_tensor_slices((test_X, test_y))

In [61]:
# wav 불러오기
def load_wav_for_map(filename, label):
    return load_wav_16k_mono(filename), label

train_ds = train_ds.map(load_wav_for_map)
valid_ds = valid_ds.map(load_wav_for_map)
test_ds = test_ds.map(load_wav_for_map)

In [62]:
# train_X = train_X.apply(lambda x: load_wav_16k_mono(x))
# valid_X = valid_X.apply(lambda x: load_wav_16k_mono(x))
# test_X = test_X.apply(lambda x: load_wav_16k_mono(x))

In [63]:
# yamnet 임베딩 적용
# applies the embedding extraction model to a wav data
def extract_embedding(wav_data,targets):
    scores, embeddings, spectrogram = yamnet_model(wav_data)
    num_embeddings = tf.shape(embeddings)[0]
    return (embeddings,
           tf.repeat(targets, num_embeddings))

# extract embedding
train_ds = train_ds.map(extract_embedding).unbatch()
valid_ds = valid_ds.map(extract_embedding).unbatch()
test_ds = test_ds.map(extract_embedding).unbatch()

In [64]:
# def extract_embedding2(wav_data):
#   scores,embeddings, spectrogram = yamnet_model(wav_data)
#   return embeddings

In [65]:
# train_X = train_X.apply(lambda x: extract_embedding2(x))
# valid_X = valid_X.apply(lambda x: extract_embedding2(x))
# test_X = test_X.apply(lambda x: extract_embedding2(x))

In [66]:
# 배치 사이즈 적용
train_ds = train_ds.cache().shuffle(1000).batch(64).prefetch(tf.data.AUTOTUNE)
valid_ds = valid_ds.cache().batch(64).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.cache().batch(64).prefetch(tf.data.AUTOTUNE)

In [67]:
# 모델링
my_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1024), dtype=tf.float32,
                          name='input_embedding'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(len(my_classes), activation = 'softmax')
    ], name='my_model')

my_model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 512)               524800    
                                                                 
 dense_3 (Dense)             (None, 6)                 3078      
                                                                 
Total params: 527,878
Trainable params: 527,878
Non-trainable params: 0
_________________________________________________________________


In [68]:
my_model.compile(loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                 optimizer="adam",
                metrics = ['acc'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                            patience=3,
                                            restore_best_weights=True)

In [69]:
history = my_model.fit(train_ds,epochs=20, validation_data = valid_ds)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/keras/backend.py:5583: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits, "Softmax", "sparse_categorical_crossentropy"


6311/6311 [==============================] - 730s 115ms/step - loss: 1.3029 - acc: 0.5249 - val_loss: 1.9746 - val_acc: 0.2883
Epoch 2/20
6311/6311 [==============================] - 18s 3ms/step - loss: 1.3210 - acc: 0.5203 - val_loss: 1.8967 - val_acc: 0.2824
Epoch 3/20
6311/6311 [==============================] - 18s 3ms/step - loss: 1.3286 - acc: 0.5096 - val_loss: 2.0249 - val_acc: 0.3092
Epoch 4/20
6311/6311 [==============================] - 18s 3ms/step - loss: 1.3190 - acc: 0.5101 - val_loss: 1.9357 - val_acc: 0.3167
Epoch 5/20
6311/6311 [==============================] - 18s 3ms/step - loss: 1.3091 - acc: 0.5142 - val_loss: 2.1365 - val_acc: 0.3324
Epoch 6/20
6311/6311 [==============================] - 18s 3ms/step - loss: 1.2918 - acc: 0.5169 - val_loss: 2.1436 - val_acc: 0.3178
Epoch 7/20
6311/6311 [==============================] - 18s 3ms/step - loss: 1.2834 - acc: 0.5237 - val_loss: 2.2745 - val_acc: 0.3106
Epoch 8/20
6311/6311 [==============================] - 18s 3ms

In [70]:
# loss, accuracy = my_model.evaluate()
# print(loss, accuracy)